# 🎥 Live ID Card Extraction - Trained Detection

This notebook uses your existing ID card dataset to **train** the detection system, making it much more accurate for live capture.

## Workflow:
1. **Train on your dataset** - Learn what your ID cards look like
2. **Live detection** - Use trained model for real-time capture
3. **Extract information** - Name, Department, Moodle ID

## Setup and Imports

In [36]:
import cv2
import numpy as np
import easyocr
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
from pathlib import Path
from datetime import datetime
import torch
from glob import glob

# Check GPU
use_gpu = torch.cuda.is_available()
print(f"GPU Available: {use_gpu}")

# Initialize EasyOCR
print("Initializing EasyOCR...")
reader = easyocr.Reader(['en'], gpu=use_gpu, verbose=False)
print("✅ Ready!")

GPU Available: True
Initializing EasyOCR...
✅ Ready!
✅ Ready!


## Configuration

In [37]:
# Paths
DATA_DIR = Path("data")
OUTPUT_DIR = Path("output/live_capture")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
PHOTO_DIR = OUTPUT_DIR / "photos"
PHOTO_DIR.mkdir(exist_ok=True)

# Camera settings
CAMERA_INDEX = 0
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720

# Region definitions
REGIONS = {
    'ignore_top': (0, 0, 1, 0.24),
    'photo': (0.20, 0.25, 0.55, 0.65),
    'name': (0.10, 0.68, 0.90, 0.76),
    'department': (0.10, 0.76, 0.90, 0.84),
    'moodle_id': (0.10, 0.92, 0.70, 1.0),
}

# Department mapping
DEPARTMENT_MAPPING = {
    'COMP': 'Computer Engineering', 'COMPUTER': 'Computer Engineering',
    'IT': 'Information Technology', 'INFORMATION TECHNOLOGY': 'Information Technology',
    'AIML': 'AI & Machine Learning', 'AI ML': 'AI & Machine Learning',
    'DS': 'Data Science', 'DATA SCIENCE': 'Data Science',
    'EXTC': 'Electronics & Telecom', 'ELECTRONICS': 'Electronics & Telecom',
    'MECH': 'Mechanical', 'MECHANICAL': 'Mechanical',
}

# Forbidden words for names
NAME_FORBIDDEN_WORDS = [
    'principal', 'dr', 'prof', 'college', 'institute', 'technology',
    'computer', 'information', 'engineering', 'mechanical', 'electronics',
    'aiml', 'data', 'science', 'id', 'no', 'moodle', 'student', 'card'
]

ENGINEERING_KEYWORDS = [
    'computer', 'information', 'technology', 'engineering',
    'mechanical', 'electronics', 'aiml', 'data', 'science'
]

print(f"✅ Configuration loaded!")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Output directory: {OUTPUT_DIR}")

✅ Configuration loaded!
📁 Data directory: data
📁 Output directory: output\live_capture


## Step 1: Train ID Card Detector on Your Dataset

In [38]:
def analyze_dataset_for_id_cards():
    """
    Analyze your dataset to learn what ID cards look like
    Returns: Statistics and reference features for detection
    """
    print("="*70)
    print("🔍 ANALYZING YOUR ID CARD DATASET")
    print("="*70)
    
    image_files = list(DATA_DIR.glob("*.jpg"))
    print(f"\n📊 Found {len(image_files)} images in dataset\n")
    
    # Statistics
    aspect_ratios = []
    card_colors = []
    card_features = []
    
    print("Analyzing each image...")
    for i, img_path in enumerate(image_files[:10], 1):  # Sample first 10
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        
        h, w = img.shape[:2]
        aspect_ratio = w / h if h > 0 else 0
        aspect_ratios.append(aspect_ratio)
        
        # Dominant color
        avg_color = np.mean(img, axis=(0, 1))
        card_colors.append(avg_color)
        
        print(f"  {i}. {img_path.name[:30]:30} - {w}x{h} (ratio: {aspect_ratio:.2f})")
    
    # Calculate statistics
    avg_ratio = np.mean(aspect_ratios) if aspect_ratios else 1.5
    std_ratio = np.std(aspect_ratios) if aspect_ratios else 0.2
    
    print(f"\n📐 ID Card Statistics:")
    print(f"   Average aspect ratio: {avg_ratio:.2f}")
    print(f"   Std deviation: {std_ratio:.2f}")
    print(f"   Expected range: {avg_ratio - 2*std_ratio:.2f} to {avg_ratio + 2*std_ratio:.2f}")
    
    return {
        'avg_ratio': avg_ratio,
        'std_ratio': std_ratio,
        'min_ratio': avg_ratio - 2*std_ratio,
        'max_ratio': avg_ratio + 2*std_ratio,
        'sample_count': len(aspect_ratios)
    }

# Train the detector
card_stats = analyze_dataset_for_id_cards()
print("\n✅ Training complete! Detector is ready.")

🔍 ANALYZING YOUR ID CARD DATASET

📊 Found 0 images in dataset

Analyzing each image...

📐 ID Card Statistics:
   Average aspect ratio: 1.50
   Std deviation: 0.20
   Expected range: 1.10 to 1.90

✅ Training complete! Detector is ready.


## Step 2: Trained Detection Function

In [39]:
def detect_id_card_trained(frame, card_stats):
    """
    ULTRA-LENIENT GRAYSCALE ID CARD DETECTION
    Converts to grayscale for better edge detection and contrast
    """
    
    # === CONVERT TO GRAYSCALE FIRST ===
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE for better contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    
    # === STEP 1: Enhanced edge detection ===
    bilateral = cv2.bilateralFilter(gray, 11, 80, 80)
    
    # Multiple Canny thresholds - VERY LENIENT
    edges1 = cv2.Canny(bilateral, 30, 100)
    edges2 = cv2.Canny(bilateral, 50, 150)
    edges3 = cv2.Canny(gray, 40, 120)
    
    # Combine edges
    edges_combined = cv2.bitwise_or(edges1, edges2)
    edges_combined = cv2.bitwise_or(edges_combined, edges3)
    
    # Morphological closing - STRONG
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    kernel_rect = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    
    edges_closed = cv2.morphologyEx(edges_combined, cv2.MORPH_CLOSE, kernel_ellipse, iterations=5)
    edges_dilated = cv2.dilate(edges_closed, kernel_rect, iterations=3)
    
    # === STEP 2: Find contours ===
    contours, _ = cv2.findContours(edges_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    frame_h, frame_w = frame.shape[:2]
    frame_area = frame_h * frame_w
    
    candidates = []
    
    for contour in sorted(contours, key=cv2.contourArea, reverse=True)[:20]:
        area = cv2.contourArea(contour)
        
        # === VERY LENIENT SIZE FILTER ===
        if area < 0.03 * frame_area or area > 0.90 * frame_area:
            continue
        
        # Approximate to polygon
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        
        if len(approx) < 4:
            continue
        
        # Get bounding box
        x, y, w, h = cv2.boundingRect(contour)
        
        # === VERY LENIENT SIZE CHECK ===
        if w < 80 or h < 120:
            continue
        
        # === VERY LENIENT ASPECT RATIO - PORTRAIT ===
        aspect_ratio = w / h
        if aspect_ratio < 0.45 or aspect_ratio > 0.95:
            continue
        
        # === VERY LENIENT RECTANGULARITY ===
        rect_area = w * h
        rectangularity = area / rect_area if rect_area > 0 else 0
        if rectangularity < 0.60:
            continue
        
        # === ANALYZE GRAYSCALE VALUES ===
        card_roi = gray[y:y+h, x:x+w]
        
        # Check for mixed intensity values (white + darker sections)
        mean_intensity = np.mean(card_roi)
        std_intensity = np.std(card_roi)
        
        # Cards should have good contrast (varying intensities)
        if std_intensity < 20:  # Too uniform
            continue
        
        # === EDGE DENSITY CHECK ===
        edge_roi = edges_combined[y:y+h, x:x+w]
        edge_density = np.count_nonzero(edge_roi) / (w * h)
        
        # Should have some edges (text/photo) but not too many
        if edge_density < 0.02 or edge_density > 0.20:
            continue
        
        # === CALCULATE SCORE ===
        score = 0.0
        
        # Size score
        ideal_area_ratio = 0.25
        area_ratio = area / frame_area
        size_score = 1.0 - abs(area_ratio - ideal_area_ratio) / ideal_area_ratio
        size_score = max(0, size_score)
        score += size_score * 0.25
        
        # Aspect ratio score (ideal: 0.63)
        ideal_ar = 0.63
        ar_score = 1.0 - abs(aspect_ratio - ideal_ar) / ideal_ar
        ar_score = max(0, ar_score)
        score += ar_score * 0.20
        
        # Rectangularity score
        score += rectangularity * 0.20
        
        # Contrast score (higher std = better)
        contrast_score = min(1.0, std_intensity / 100.0)
        score += contrast_score * 0.20
        
        # Edge density score
        ideal_edge = 0.08
        edge_score = 1.0 - abs(edge_density - ideal_edge) / ideal_edge
        edge_score = max(0, edge_score)
        score += edge_score * 0.15
        
        candidates.append({
            'bbox': (x, y, w, h),
            'score': score,
            'aspect_ratio': aspect_ratio,
            'rectangularity': rectangularity,
            'area_ratio': area_ratio,
            'mean_intensity': mean_intensity,
            'std_intensity': std_intensity,
            'edge_density': edge_density
        })
    
    # === STEP 3: Select best candidate ===
    if not candidates:
        return None
    
    # VERY LOW THRESHOLD
    candidates = [c for c in candidates if c['score'] >= 0.20]
    
    if not candidates:
        return None
    
    # Sort by score
    best = max(candidates, key=lambda x: x['score'])
    
    print(f"✓ Card detected! Score: {best['score']:.3f}, AR: {best['aspect_ratio']:.2f}, "
          f"Intensity: {best['mean_intensity']:.1f}±{best['std_intensity']:.1f}, "
          f"Edges: {best['edge_density']:.3f}")
    
    return best['bbox']


print("✓ Ultra-lenient grayscale detection function loaded!")
print("Features:")
print("  - Grayscale conversion with CLAHE")
print("  - Very low thresholds (3-90% size, 0.45-0.95 AR)")
print("  - Intensity-based filtering")
print("  - Score threshold: 0.20")

✓ Ultra-lenient grayscale detection function loaded!
Features:
  - Grayscale conversion with CLAHE
  - Very low thresholds (3-90% size, 0.45-0.95 AR)
  - Intensity-based filtering
  - Score threshold: 0.20


## Helper Functions

In [40]:
def get_region(img, region_coords):
    h, w = img.shape[:2]
    x1, y1, x2, y2 = region_coords
    x1_px = max(0, int(x1 * w))
    y1_px = max(0, int(y1 * h))
    x2_px = min(w, int(x2 * w))
    y2_px = min(h, int(y2 * h))
    return img[y1_px:y2_px, x1_px:x2_px], (x1_px, y1_px, x2_px, y2_px)

def extract_text(img_region):
    try:
        if img_region.size == 0:
            return ''
        results = reader.readtext(img_region, detail=0, paragraph=False)
        return ' '.join(results).strip() if results else ''
    except:
        return ''

def clean_name(text):
    if not text:
        return ''
    text_clean = re.sub(r'\d+', '', text)
    text_clean = re.sub(r'[^a-zA-Z\s]', '', text_clean)
    words = [w.title() for w in text_clean.split() if len(w) >= 2 and 
             w.lower() not in NAME_FORBIDDEN_WORDS and
             not any(kw in w.lower() for kw in ENGINEERING_KEYWORDS)]
    return ' '.join(words) if 2 <= len(words) <= 4 else ''

def normalize_department(text):
    if not text:
        return ''
    text_upper = text.upper().strip()
    text_upper = re.sub(r'\d+', '', text_upper)
    text_upper = text_upper.replace('ID NO', '').replace('ID', '').replace('NO', '')
    text_upper = re.sub(r'[^A-Z\s&]', '', text_upper).strip()
    
    for key, value in DEPARTMENT_MAPPING.items():
        if key in text_upper:
            return value
    
    if any(kw in text_upper.lower() for kw in ENGINEERING_KEYWORDS):
        return text_upper.title()
    return ''

def extract_moodle_id(text):
    text = text.replace('ID NO', '').replace('ID', '').replace('NO', '')
    match = re.search(r'2\d{7}', text)
    return match.group(0) if match else None

print("✅ Helper functions loaded!")

✅ Helper functions loaded!


## Extraction Functions

In [41]:
def extract_from_id_card(card_img):
    """Extract all information from ID card"""
    name_region, name_coords = get_region(card_img, REGIONS['name'])
    name_text = extract_text(name_region)
    name = clean_name(name_text)
    
    dept_region, dept_coords = get_region(card_img, REGIONS['department'])
    dept_text = extract_text(dept_region)
    department = normalize_department(dept_text)
    
    id_region, id_coords = get_region(card_img, REGIONS['moodle_id'])
    id_text = extract_text(id_region)
    moodle_id = extract_moodle_id(id_text)
    
    photo_region, photo_coords = get_region(card_img, REGIONS['photo'])
    
    return {
        'name': name, 'name_text': name_text, 'name_coords': name_coords,
        'department': department, 'dept_text': dept_text, 'dept_coords': dept_coords,
        'moodle_id': moodle_id, 'id_text': id_text, 'id_coords': id_coords,
        'photo': photo_region, 'photo_coords': photo_coords
    }

def draw_overlay(frame, card_bbox, info):
    """Draw complete overlay with regions and info"""
    overlay = frame.copy()
    
    # Draw card box
    if card_bbox:
        x, y, w, h = card_bbox
        cv2.rectangle(overlay, (x, y), (x+w, y+h), (0, 255, 0), 3)
        cv2.putText(overlay, "ID CARD", (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Draw regions on card
        if info:
            colors = {'photo': (0,255,0), 'name': (0,0,255), 
                     'dept': (255,0,0), 'id': (0,255,255)}
            
            for region_name, color in colors.items():
                coords_key = f"{region_name}_coords" if region_name != 'id' else 'id_coords'
                if coords_key in info:
                    rx1, ry1, rx2, ry2 = info[coords_key]
                    cv2.rectangle(overlay, (x+rx1, y+ry1), (x+rx2, y+ry2), color, 2)
                    cv2.putText(overlay, region_name.upper()[:4], (x+rx1, y+ry1-5),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    # Info panel
    cv2.rectangle(overlay, (10, 10), (600, 200), (0, 0, 0), -1)
    cv2.rectangle(overlay, (10, 10), (600, 200), (0, 255, 0), 2)
    cv2.putText(overlay, "LIVE ID CAPTURE", (20, 40),
               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    
    y_pos = 75
    if info:
        for field, label in [('name', 'Name'), ('department', 'Dept'), ('moodle_id', 'ID')]:
            value = info.get(field, '')
            status = "✓" if value else "✗"
            color = (0, 255, 0) if value else (0, 0, 255)
            cv2.putText(overlay, f"{status} {label}: {value if value else 'Not detected'}",
                       (20, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            y_pos += 35
    else:
        cv2.putText(overlay, "Hold ID card in frame...", (20, y_pos),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
    
    # Controls
    y_pos = frame.shape[0] - 50
    cv2.rectangle(overlay, (10, y_pos-10), (650, frame.shape[0]-10), (0, 0, 0), -1)
    cv2.putText(overlay, "[SPACE] Capture | [Q] Quit | [S] Save | [D] Debug",
               (20, y_pos + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    return cv2.addWeighted(overlay, 0.85, frame, 0.15, 0)

print("✅ Extraction functions loaded!")

✅ Extraction functions loaded!


## 🚀 Live Capture System

In [42]:
def live_id_card_detection():
    """ULTRA-FAST GRAYSCALE LIVE DETECTION - Press SPACE to capture"""
    
    print("="*70)
    print("GRAYSCALE ID CARD LIVE DETECTION")
    print("="*70)
    print("\nControls:")
    print("  SPACE - Detect & process ID card")
    print("  Q     - Quit")
    print("\nTips for best detection:")
    print("  - Hold card FLAT and CENTERED")
    print("  - Good lighting, no shadows")
    print("  - Card should fill 20-50% of frame")
    print("  - Portrait orientation (vertical)")
    print("="*70)
    
    cap = cv2.VideoCapture(CAMERA_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
    
    if not cap.isOpened():
        print("❌ Cannot open webcam!")
        return
    
    print(f"\n✓ Webcam opened: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")
    
    last_detection = None
    frame_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            display_frame = frame.copy()
            
            # Show grayscale preview in corner
            gray_preview = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_preview = cv2.resize(gray_preview, (200, 150))
            gray_preview_bgr = cv2.cvtColor(gray_preview, cv2.COLOR_GRAY2BGR)
            display_frame[10:160, 10:210] = gray_preview_bgr
            cv2.rectangle(display_frame, (10, 10), (210, 160), (0, 255, 0), 2)
            cv2.putText(display_frame, "GRAYSCALE", (15, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Show last detection
            if last_detection:
                x, y, w, h = last_detection
                cv2.rectangle(display_frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
                cv2.putText(display_frame, "LAST DETECTED", (x, y-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Instructions
            cv2.putText(display_frame, "Press SPACE to detect ID card", 
                       (20, frame.shape[0] - 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            
            cv2.imshow('ID Card Detection (Grayscale)', display_frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                print("\n👋 Exiting...")
                break
            
            elif key == ord(' '):
                print(f"\n{'='*70}")
                print(f"FRAME {frame_count} - PROCESSING...")
                print('='*70)
                
                import time
                start_time = time.time()
                
                # Detect card
                bbox = detect_id_card_trained(frame, card_stats)
                
                if bbox is None:
                    print("❌ No ID card detected")
                    print("   Try:")
                    print("   - Better lighting")
                    print("   - Hold card flatter")
                    print("   - Center the card")
                    print("   - Make card fill 20-50% of frame")
                    continue
                
                last_detection = bbox
                x, y, w, h = bbox
                
                print(f"✅ Card detected at: x={x}, y={y}, w={w}, h={h}")
                
                # Crop card
                card_img = frame[y:y+h, x:x+w]
                
                # Process with EasyOCR
                print("\n📝 Running OCR...")
                results = reader.readtext(card_img)
                
                # Extract fields
                all_text = ' '.join([text for (_, text, _) in results])
                print(f"\n📄 Extracted text: {all_text}")
                
                # Try to identify name, dept, ID
                name = ""
                dept = ""
                moodle_id = ""
                
                for (bbox_ocr, text, conf) in results:
                    text_clean = text.strip().upper()
                    
                    # Department matching
                    for key, dept_name in DEPARTMENT_MAPPING.items():
                        if key in text_clean:
                            dept = dept_name
                            break
                    
                    # Moodle ID matching (8 digits starting with 2)
                    import re
                    id_match = re.search(r'2\d{7}', text_clean)
                    if id_match:
                        moodle_id = id_match.group()
                    
                    # Name (heuristic: longest alphabetic string)
                    if len(text.split()) >= 2 and text.isalpha() and len(text) > len(name):
                        name = text
                
                elapsed = time.time() - start_time
                
                print(f"\n{'='*70}")
                print("RESULTS:")
                print(f"  Name:       {name or 'Not found'}")
                print(f"  Department: {dept or 'Not found'}")
                print(f"  Moodle ID:  {moodle_id or 'Not found'}")
                print(f"  Time:       {elapsed:.2f}s")
                print('='*70)
                
                # Show detected card
                cv2.imshow('Detected Card', card_img)
                cv2.waitKey(2000)
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("\n✓ Cleanup complete")


print("✓ Live detection function loaded!")
print("Run: live_id_card_detection()")

✓ Live detection function loaded!
Run: live_id_card_detection()


## ▶️ RUN THIS - Start Live Capture

In [43]:
# Start the trained live capture system
run_live_capture(card_stats)

🎥 STARTING TRAINED LIVE CAPTURE

Controls: [SPACE] Capture | [Q] Quit | [S] Save | [D] Debug

🎬 Opening camera...

✅ Camera ready: 1280x720

💡 Position your ID card horizontally in the frame
   Wait for green box and all ✓ marks before pressing SPACE
   Press D for debug mode to see what's being detected

✅ Camera ready: 1280x720

💡 Position your ID card horizontally in the frame
   Wait for green box and all ✓ marks before pressing SPACE
   Press D for debug mode to see what's being detected


⚠️  No card detected - try:
   - Move card closer/farther
   - Ensure card is flat and horizontal
   - Press D to see debug view

⚠️  No card detected - try:
   - Move card closer/farther
   - Ensure card is flat and horizontal
   - Press D to see debug view

⚠️  No card detected - try:
   - Move card closer/farther
   - Ensure card is flat and horizontal
   - Press D to see debug view

⚠️  No card detected - try:
   - Move card closer/farther
   - Ensure card is flat and horizontal
   - Press D

## 🧪 Test Detection on Single Image (Optional)

Run this to test detection on one of your dataset images before trying live capture.

In [44]:
# Test detection on a sample image
test_img_path = DATA_DIR / "abhishek.jpg"  # Change to any image name
if test_img_path.exists():
    test_img = cv2.imread(str(test_img_path))
    
    print(f"Testing detection on: {test_img_path.name}")
    print(f"Image size: {test_img.shape[1]}x{test_img.shape[0]}")
    
    # Try detection
    bbox = detect_id_card_trained(test_img, card_stats)
    
    if bbox:
        x, y, w, h = bbox
        ar = w/h if h > 0 else 0
        print(f"✅ DETECTED! Box: ({x}, {y}, {w}, {h})")
        print(f"   Aspect ratio: {ar:.2f}")
        print(f"   Size: {w}x{h}")
        
        # Draw on image
        result = test_img.copy()
        cv2.rectangle(result, (x, y), (x+w, y+h), (0, 255, 0), 3)
        
        # Show
        plt.figure(figsize=(12, 6))
        plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        plt.title(f"Detection Test - {test_img_path.name}")
        plt.axis('off')
        plt.show()
    else:
        print("❌ NOT DETECTED - Card not found in image")
        print("   Try adjusting detection parameters or use debug mode")
else:
    print(f"Image not found: {test_img_path}")

Image not found: data\abhishek.jpg


## View Captured Data

In [45]:
# Run the ultra-lenient grayscale detection!
live_id_card_detection()

GRAYSCALE ID CARD LIVE DETECTION

Controls:
  SPACE - Detect & process ID card
  Q     - Quit

Tips for best detection:
  - Hold card FLAT and CENTERED
  - Good lighting, no shadows
  - Card should fill 20-50% of frame
  - Portrait orientation (vertical)

✓ Webcam opened: 1280x720

✓ Webcam opened: 1280x720

👋 Exiting...

👋 Exiting...

✓ Cleanup complete

✓ Cleanup complete


# 🔐 Face Verification System

**Complete ID Card Face Verification against Database**

Features:
- Webcam capture of ID card
- Extract student photo from ID card (APSIT layout)
- Generate face embeddings using DeepFace + Facenet
- Match against database (students.csv)
- Cosine similarity with threshold 0.6
- Side-by-side visualization

In [46]:
# Install required packages
%pip install deepface scikit-learn tf-keras -q

# Import additional libraries for face verification
from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity
import json
import time

# Configuration
FACE_MODEL = 'Facenet'  # High accuracy, 512-d embeddings
SIMILARITY_THRESHOLD = 0.6  # Cosine similarity threshold
DB_FILE = DATA_DIR / "students.csv"
SAMPLE_FACE = DATA_DIR / "sample_face.jpg"  # For testing
CROPPED_OUTPUT = OUTPUT_DIR / "cropped_photo.jpg"

# ID card photo crop coordinates (APSIT layout)
ID_PHOTO_CROP = {
    'x1': 0.25,  # 25% from left
    'y1': 0.35,  # 35% from top
    'x2': 0.55,  # 55% from left
    'y2': 0.65   # 65% from top
}

print("✅ Face verification imports loaded!")
print(f"   Model: {FACE_MODEL}")
print(f"   Threshold: {SIMILARITY_THRESHOLD}")
print(f"   Database: {DB_FILE}")
print(f"   GPU Available: {use_gpu}")


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


25-10-04 03:07:56 - Directory C:\Users\DELL\.deepface has been created
25-10-04 03:07:56 - Directory C:\Users\DELL\.deepface\weights has been created
25-10-04 03:07:56 - Directory C:\Users\DELL\.deepface has been created
25-10-04 03:07:56 - Directory C:\Users\DELL\.deepface\weights has been created
✅ Face verification imports loaded!
   Model: Facenet
   Threshold: 0.6
   Database: data\students.csv
   GPU Available: True
✅ Face verification imports loaded!
   Model: Facenet
   Threshold: 0.6
   Database: data\students.csv
   GPU Available: True


## Core Face Verification Functions

In [47]:
def capture_id_card():
    """
    Capture ID card image from webcam
    Returns: Grayscale denoised image frame or None
    """
    print("\n" + "="*70)
    print("📷 CAPTURING ID CARD FROM WEBCAM")
    print("="*70)
    print("Instructions:")
    print("  - Hold ID card flat and centered")
    print("  - Press SPACE to capture")
    print("  - Press Q to cancel")
    print("="*70)
    
    cap = cv2.VideoCapture(CAMERA_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
    
    if not cap.isOpened():
        print("❌ Cannot open webcam!")
        return None
    
    captured_frame = None
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Convert to grayscale for preview
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            
            # Show both color and grayscale
            display = frame.copy()
            small_gray = cv2.resize(gray_bgr, (200, 150))
            display[10:160, 10:210] = small_gray
            cv2.rectangle(display, (10, 10), (210, 160), (0, 255, 0), 2)
            cv2.putText(display, "GRAYSCALE", (15, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            cv2.putText(display, "Press SPACE to capture ID card", 
                       (20, frame.shape[0] - 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            
            cv2.imshow('Capture ID Card', display)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                print("❌ Capture cancelled")
                break
            elif key == ord(' '):
                # Convert to grayscale
                gray_captured = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                # Apply denoising (dust/noise reduction)
                print("🧹 Applying image cleaning (denoising)...")
                denoised = cv2.fastNlMeansDenoising(
                    gray_captured,
                    h=10,  # Filter strength
                    templateWindowSize=7,
                    searchWindowSize=21
                )
                
                # Save grayscale denoised image
                output_path = OUTPUT_DIR / "live_capture" / "id_card.jpg"
                cv2.imwrite(str(output_path), denoised)
                print(f"✅ Grayscale denoised image saved to: {output_path}")
                
                # Return as BGR for compatibility with extract_photo
                captured_frame = cv2.cvtColor(denoised, cv2.COLOR_GRAY2BGR)
                print("✅ ID card captured!")
                break
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print("🧹 All windows closed")
    
    return captured_frame


def extract_photo(img):
    """
    Extract student photo from ID card using APSIT layout
    Args:
        img: BGR image of ID card
    Returns:
        cropped photo (BGR) or None
    """
    if img is None:
        return None
    
    h, w = img.shape[:2]
    
    # Calculate crop coordinates
    x1 = int(ID_PHOTO_CROP['x1'] * w)
    y1 = int(ID_PHOTO_CROP['y1'] * h)
    x2 = int(ID_PHOTO_CROP['x2'] * w)
    y2 = int(ID_PHOTO_CROP['y2'] * h)
    
    # Ensure valid bounds
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)
    
    if x2 <= x1 or y2 <= y1:
        print("❌ Invalid crop coordinates")
        return None
    
    cropped = img[y1:y2, x1:x2]
    
    print(f"✅ Photo extracted: {cropped.shape[1]}x{cropped.shape[0]}")
    print(f"   Crop region: ({x1}, {y1}) to ({x2}, {y2})")
    
    # Save cropped photo
    cv2.imwrite(str(CROPPED_OUTPUT), cropped)
    print(f"   Saved to: {CROPPED_OUTPUT}")
    
    return cropped


def get_embedding(img_path_or_array):
    """
    Generate face embedding using DeepFace + Facenet
    Args:
        img_path_or_array: Path to image or numpy array (BGR)
    Returns:
        numpy array of embedding (512-d) or None
    """
    try:
        # DeepFace can accept path or numpy array
        if isinstance(img_path_or_array, (str, Path)):
            img_input = str(img_path_or_array)
        else:
            # Convert BGR to RGB for DeepFace
            img_input = cv2.cvtColor(img_path_or_array, cv2.COLOR_BGR2RGB)
        
        # Generate embedding
        embedding_obj = DeepFace.represent(
            img_path=img_input,
            model_name=FACE_MODEL,
            enforce_detection=False,  # Don't fail if face not detected
            detector_backend='opencv'
        )
        
        # Extract embedding array
        if isinstance(embedding_obj, list) and len(embedding_obj) > 0:
            embedding = np.array(embedding_obj[0]['embedding'])
        else:
            embedding = np.array(embedding_obj['embedding'])
        
        print(f"✅ Embedding generated: shape {embedding.shape}")
        return embedding
    
    except Exception as e:
        print(f"❌ Failed to generate embedding: {e}")
        return None


def match_embedding(query_embedding, db_embeddings, db_df):
    """
    Match query embedding against database
    Args:
        query_embedding: numpy array (512-d)
        db_embeddings: list of numpy arrays
        db_df: pandas DataFrame with student records
    Returns:
        (best_match_index, similarity_score) or (None, 0)
    """
    if query_embedding is None or len(db_embeddings) == 0:
        return None, 0
    
    # Reshape for sklearn
    query_emb = query_embedding.reshape(1, -1)
    
    best_score = 0
    best_idx = None
    
    for idx, db_emb in enumerate(db_embeddings):
        if db_emb is None:
            continue
        
        db_emb_reshaped = db_emb.reshape(1, -1)
        
        # Calculate cosine similarity
        similarity = cosine_similarity(query_emb, db_emb_reshaped)[0][0]
        
        if similarity > best_score:
            best_score = similarity
            best_idx = idx
    
    print(f"\n🔍 Best match: similarity = {best_score:.4f}")
    
    if best_score >= SIMILARITY_THRESHOLD:
        print(f"✅ MATCH FOUND! (threshold: {SIMILARITY_THRESHOLD})")
        return best_idx, best_score
    else:
        print(f"❌ No match (below threshold {SIMILARITY_THRESHOLD})")
        return None, best_score


print("✅ Face verification functions loaded!")
print("Functions:")
print("  - capture_id_card() → captures from webcam")
print("  - extract_photo(img) → crops student photo")
print("  - get_embedding(img) → generates 512-d embedding")
print("  - match_embedding(query, db, df) → finds best match")

✅ Face verification functions loaded!
Functions:
  - capture_id_card() → captures from webcam
  - extract_photo(img) → crops student photo
  - get_embedding(img) → generates 512-d embedding
  - match_embedding(query, db, df) → finds best match


## Database Management

In [48]:
def load_database(db_path):
    """
    Load student database from CSV
    Columns: moodle_id, name, department, embedding (JSON string)
    Returns: DataFrame, list of embeddings
    """
    print(f"\n📂 Loading database: {db_path}")
    
    if not db_path.exists():
        print(f"❌ Database not found: {db_path}")
        print("   Creating sample database...")
        
        # Create sample database
        sample_data = {
            'moodle_id': ['20210001', '20210002', '20210003'],
            'name': ['John Doe', 'Jane Smith', 'Bob Johnson'],
            'department': ['Computer Engineering', 'Information Technology', 'Computer Engineering'],
            'embedding': ['[]', '[]', '[]']  # Empty embeddings
        }
        df = pd.DataFrame(sample_data)
        df.to_csv(db_path, index=False)
        print(f"✅ Sample database created: {db_path}")
        return df, []
    
    # Load CSV
    df = pd.read_csv(db_path)
    print(f"✅ Loaded {len(df)} records")
    
    # Parse embeddings
    embeddings = []
    for idx, row in df.iterrows():
        try:
            emb_str = row['embedding']
            if pd.isna(emb_str) or emb_str == '[]' or emb_str == '':
                embeddings.append(None)
            else:
                emb_array = np.array(json.loads(emb_str))
                embeddings.append(emb_array)
        except Exception as e:
            print(f"   Warning: Failed to parse embedding for row {idx}: {e}")
            embeddings.append(None)
    
    valid_count = sum(1 for e in embeddings if e is not None)
    print(f"   Valid embeddings: {valid_count}/{len(embeddings)}")
    
    return df, embeddings


def add_to_database(db_path, moodle_id, name, department, embedding):
    """
    Add or update student record in database
    Args:
        db_path: Path to CSV file
        moodle_id: Student ID
        name: Student name
        department: Department name
        embedding: numpy array (512-d)
    """
    # Load existing database
    if db_path.exists():
        df = pd.read_csv(db_path)
    else:
        df = pd.DataFrame(columns=['moodle_id', 'name', 'department', 'embedding'])
    
    # Convert embedding to JSON string
    emb_str = json.dumps(embedding.tolist()) if embedding is not None else '[]'
    
    # Check if student already exists
    existing = df[df['moodle_id'] == moodle_id]
    
    if len(existing) > 0:
        # Update existing record
        df.loc[df['moodle_id'] == moodle_id, 'name'] = name
        df.loc[df['moodle_id'] == moodle_id, 'department'] = department
        df.loc[df['moodle_id'] == moodle_id, 'embedding'] = emb_str
        print(f"✅ Updated record for {moodle_id}")
    else:
        # Add new record
        new_row = pd.DataFrame([{
            'moodle_id': moodle_id,
            'name': name,
            'department': department,
            'embedding': emb_str
        }])
        df = pd.concat([df, new_row], ignore_index=True)
        print(f"✅ Added new record for {moodle_id}")
    
    # Save database
    df.to_csv(db_path, index=False)
    print(f"   Database saved: {db_path}")


print("✅ Database functions loaded!")
print("Functions:")
print("  - load_database(path) → loads CSV and embeddings")
print("  - add_to_database(path, id, name, dept, emb) → adds/updates record")

✅ Database functions loaded!
Functions:
  - load_database(path) → loads CSV and embeddings
  - add_to_database(path, id, name, dept, emb) → adds/updates record


## 🚀 Complete Verification Pipeline

In [49]:
def verify_id_card_complete():
    """
    Complete face verification pipeline:
    1. Capture ID card from webcam
    2. Extract student photo
    3. Generate embedding
    4. Match against database
    5. Display results
    """
    print("="*70)
    print("🔐 COMPLETE ID CARD FACE VERIFICATION SYSTEM")
    print("="*70)
    
    start_time = time.time()
    
    # Step 1: Capture ID card
    print("\n[1/6] Capturing ID card from webcam...")
    id_card_img = capture_id_card()
    
    if id_card_img is None:
        print("❌ No image captured")
        return
    
    # Convert to grayscale for detection
    gray = cv2.cvtColor(id_card_img, cv2.COLOR_BGR2GRAY)
    print("✅ Converted to grayscale for processing")
    
    # Step 2: Extract student photo
    print("\n[2/6] Extracting student photo from ID card...")
    cropped_photo = extract_photo(id_card_img)
    
    if cropped_photo is None:
        print("❌ Failed to extract photo")
        return
    
    # Display cropped photo
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(cropped_photo, cv2.COLOR_BGR2RGB))
    plt.title("Cropped ID Card Photo")
    plt.axis('off')
    plt.show()
    
    # Step 3: Generate embedding for cropped photo
    print("\n[3/6] Generating face embedding for ID card photo...")
    id_embedding = get_embedding(cropped_photo)
    
    if id_embedding is None:
        print("❌ Failed to generate embedding")
        return
    
    # Step 4: Load database
    print("\n[4/6] Loading student database...")
    db_df, db_embeddings = load_database(DB_FILE)
    
    # Step 5: Match with sample face (if exists)
    if SAMPLE_FACE.exists():
        print("\n[5/6] Comparing with sample face...")
        sample_embedding = get_embedding(SAMPLE_FACE)
        
        if sample_embedding is not None:
            sample_query = id_embedding.reshape(1, -1)
            sample_db = sample_embedding.reshape(1, -1)
            sample_similarity = cosine_similarity(sample_query, sample_db)[0][0]
            
            print(f"   Similarity with sample: {sample_similarity:.4f}")
            
            if sample_similarity >= SIMILARITY_THRESHOLD:
                print(f"   ✅ MATCH with sample face!")
            else:
                print(f"   ❌ No match with sample (threshold: {SIMILARITY_THRESHOLD})")
    else:
        print("\n[5/6] Sample face not found, skipping...")
    
    # Step 6: Match against database
    print("\n[6/6] Matching against database...")
    best_idx, best_score = match_embedding(id_embedding, db_embeddings, db_df)
    
    elapsed_time = time.time() - start_time
    
    # Display results
    print("\n" + "="*70)
    print("📊 VERIFICATION RESULTS")
    print("="*70)
    
    if best_idx is not None:
        match_record = db_df.iloc[best_idx]
        print("✅ AUTHORIZED ID CARD")
        print(f"\n   Student Details:")
        print(f"   Moodle ID:   {match_record['moodle_id']}")
        print(f"   Name:        {match_record['name']}")
        print(f"   Department:  {match_record['department']}")
        print(f"   Similarity:  {best_score:.4f}")
    else:
        print("❌ UNAUTHORIZED ID")
        print(f"   No matching student found in database")
        print(f"   Best similarity: {best_score:.4f} (threshold: {SIMILARITY_THRESHOLD})")
    
    print(f"\n⏱️  Total time: {elapsed_time:.2f}s")
    print("="*70)
    
    # Visualize side-by-side comparison
    if best_idx is not None and SAMPLE_FACE.exists():
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        # ID card photo
        axes[0].imshow(cv2.cvtColor(cropped_photo, cv2.COLOR_BGR2RGB))
        axes[0].set_title("ID Card Photo (Cropped)")
        axes[0].axis('off')
        
        # Sample/matched photo
        if SAMPLE_FACE.exists():
            sample_img = cv2.imread(str(SAMPLE_FACE))
            axes[1].imshow(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))
            axes[1].set_title(f"Database Match\n{db_df.iloc[best_idx]['name']}\nSimilarity: {best_score:.4f}")
        else:
            axes[1].text(0.5, 0.5, 'No sample image', ha='center', va='center')
            axes[1].set_title("Database Match")
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return {
        'authorized': best_idx is not None,
        'match_index': best_idx,
        'similarity': best_score,
        'processing_time': elapsed_time
    }


print("✅ Complete verification pipeline ready!")
print("\nRun: verify_id_card_complete()")
print("\nThis will:")
print("  1. Capture ID card from webcam")
print("  2. Extract & save student photo")
print("  3. Generate face embedding")
print("  4. Compare with sample face")
print("  5. Match against database")
print("  6. Display results & visualization")

✅ Complete verification pipeline ready!

Run: verify_id_card_complete()

This will:
  1. Capture ID card from webcam
  2. Extract & save student photo
  3. Generate face embedding
  4. Compare with sample face
  5. Match against database
  6. Display results & visualization


## 🎯 Quick Test - Run Verification!

In [50]:
# Run the complete face verification system!
result = verify_id_card_complete()

🔐 COMPLETE ID CARD FACE VERIFICATION SYSTEM

[1/6] Capturing ID card from webcam...

📷 CAPTURING ID CARD FROM WEBCAM
Instructions:
  - Hold ID card flat and centered
  - Press SPACE to capture
  - Press Q to cancel
❌ Capture cancelled
❌ Capture cancelled
❌ No image captured
❌ No image captured


## 🛠️ Utility Functions - Add Students to Database

In [51]:
def register_new_student():
    """
    Register a new student in the database by capturing their ID card
    """
    print("="*70)
    print("📝 REGISTER NEW STUDENT")
    print("="*70)
    
    # Capture ID card
    print("\n[1/4] Capturing ID card...")
    id_card_img = capture_id_card()
    
    if id_card_img is None:
        print("❌ Registration cancelled")
        return
    
    # Extract photo
    print("\n[2/4] Extracting student photo...")
    cropped_photo = extract_photo(id_card_img)
    
    if cropped_photo is None:
        print("❌ Failed to extract photo")
        return
    
    # Generate embedding
    print("\n[3/4] Generating face embedding...")
    embedding = get_embedding(cropped_photo)
    
    if embedding is None:
        print("❌ Failed to generate embedding")
        return
    
    # Get student details (manual input for now)
    print("\n[4/4] Enter student details:")
    moodle_id = input("  Moodle ID (8 digits starting with 2): ").strip()
    name = input("  Name: ").strip()
    department = input("  Department: ").strip()
    
    if not moodle_id or not name or not department:
        print("❌ Invalid details provided")
        return
    
    # Add to database
    add_to_database(DB_FILE, moodle_id, name, department, embedding)
    
    print("\n" + "="*70)
    print("✅ STUDENT REGISTERED SUCCESSFULLY!")
    print(f"   Moodle ID:  {moodle_id}")
    print(f"   Name:       {name}")
    print(f"   Department: {department}")
    print("="*70)


def populate_database_from_images():
    """
    Populate database by processing ID card images from data folder
    Useful for bulk registration
    """
    print("="*70)
    print("📦 BULK DATABASE POPULATION")
    print("="*70)
    
    # Get all images
    image_files = list(DATA_DIR.glob("*.jpg"))
    print(f"\nFound {len(image_files)} images in {DATA_DIR}")
    
    if len(image_files) == 0:
        print("❌ No images found")
        return
    
    # Process first 5 images as example
    sample_images = image_files[:5]
    print(f"\nProcessing first {len(sample_images)} images as sample...")
    
    for i, img_path in enumerate(sample_images, 1):
        print(f"\n[{i}/{len(sample_images)}] Processing {img_path.name}...")
        
        try:
            # Load image
            img = cv2.imread(str(img_path))
            
            if img is None:
                print(f"   ❌ Failed to load image")
                continue
            
            # Try to detect ID card first
            bbox = detect_id_card_trained(img, card_stats)
            
            if bbox:
                x, y, w, h = bbox
                card_img = img[y:y+h, x:x+w]
            else:
                # Assume whole image is the card
                card_img = img
            
            # Extract photo
            cropped = extract_photo(card_img)
            
            if cropped is None:
                print(f"   ❌ Failed to extract photo")
                continue
            
            # Generate embedding
            embedding = get_embedding(cropped)
            
            if embedding is None:
                print(f"   ❌ Failed to generate embedding")
                continue
            
            # Use filename as ID (dummy data)
            file_stem = img_path.stem
            dummy_id = f"202100{i:02d}"
            dummy_name = file_stem.replace('_', ' ').title()
            dummy_dept = "Computer Engineering"
            
            # Add to database
            add_to_database(DB_FILE, dummy_id, dummy_name, dummy_dept, embedding)
            print(f"   ✅ Added: {dummy_name}")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    print("\n" + "="*70)
    print("✅ BULK POPULATION COMPLETE!")
    print("="*70)


print("✅ Utility functions loaded!")
print("\nFunctions:")
print("  - register_new_student() → capture & register interactively")
print("  - populate_database_from_images() → bulk register from data/ folder")

✅ Utility functions loaded!

Functions:
  - register_new_student() → capture & register interactively
  - populate_database_from_images() → bulk register from data/ folder
